In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from mpl_toolkits.axes_grid1 import ImageGrid
from os import listdir, makedirs
from os.path import join, exists, expanduser
from sklearn import preprocessing

from tqdm import tqdm
from sklearn.metrics import log_loss, accuracy_score
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50
from keras.applications import xception
from keras.applications import inception_v3
from keras.applications.vgg16 import preprocess_input, decode_predictions
from keras.models import Sequential
from keras.layers import Conv2D, Dense, MaxPooling2D, Dropout, Flatten, Activation, BatchNormalization
from keras import optimizers
from sklearn.linear_model import LogisticRegression
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [ ]:
data_dir = "./dogBreed/"

train = pd.read_csv(join(data_dir, 'labels.csv'))
# print(train.head())

# train_image = []
# for i in tqdm(range(train.shape[0])):
#     img1 = image.load_img(data_dir + '/train/' + train['id'][i], target_size=(224,224,3))
#     img = image.img_to_array(img1) / 255
#     train_image.append(img)
# X = np.array(train_image)
# print(X.shape)

# y = train['breed'].values
# le = preprocessing.LabelEncoder()
# le.fit(y)
# y_trans = le.transform(y)

# y_final = to_categorical(y_trans)
# print(y_final.shape)


In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X, y_final, test_size=0.2)
# print(X_train.shape)
# print(y_train.shape)
# print(X_test.shape)
# print(y_test.shape)

In [ ]:
model = Sequential()
model.add(Conv2D(filters=96,kernel_size=(11,11),input_shape=(224,224,3),strides=(4,4), padding='valid',kernel_initializer='random_uniform',bias_initializer='zeros'))
model.add(Activation('relu'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='valid'))

model.add(Conv2D(filters=256,kernel_size=(5,5),strides=(1,1),padding='valid',kernel_initializer='random_uniform',bias_initializer='zeros'))
model.add(Activation('relu'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2),strides=(2,2),padding='valid'))

model.add(Conv2D(filters=384,kernel_size=(3,3),strides=(1,1),padding='valid',kernel_initializer='random_uniform',bias_initializer='zeros'))
model.add(Activation('relu'))

model.add(Conv2D(filters=384,kernel_size=(3,3),strides=(1,1),padding='valid',kernel_initializer='random_uniform',bias_initializer='zeros'))
model.add(Activation('relu'))

model.add(Conv2D(filters=256,kernel_size=(3,3),strides=(1,1),padding='valid',kernel_initializer='random_uniform',bias_initializer='zeros'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='valid'))

model.add(Flatten())

model.add(Dense(4096,input_shape=(224*224*3,),kernel_initializer='random_uniform',bias_initializer='zeros'))
model.add(Activation('relu'))
model.add(Dropout(0.4))

model.add(Dense(4096, kernel_initializer='random_uniform',bias_initializer='zeros'))
model.add(Activation('relu'))
model.add(Dropout(0.4))

model.add(Dense(120, activation='softmax', kernel_initializer='random_uniform',bias_initializer='zeros'))

model.summary()

In [ ]:
opt = optimizers.SGD(lr=0.01, momentum=0.9, decay=5e-3)
model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])

In [ ]:
from keras.applications.resnet50 import preprocess_input
from keras_preprocessing.image import ImageDataGenerator

image_size = 224
datagen = ImageDataGenerator(preprocessing_function=preprocess_input, 
                             rescale=1./255.,
                             horizontal_flip=True,
                             width_shift_range = 0.2,
                             height_shift_range = 0.2,
                             validation_split=0.2)

In [ ]:
train_generator=datagen.flow_from_dataframe(
                        dataframe=train,
                        directory="./dogBreed/train/",
                        x_col="id",
                        y_col="breed",
                        has_ext=False,
                        subset="training",
                        batch_size=32,
                        seed=50,
                        shuffle=True,
                        class_mode="categorical",
                        target_size=(image_size, image_size))

In [ ]:
valid_generator=datagen.flow_from_dataframe(
                        dataframe=train,
                        directory="./dogBreed/train/",
                        x_col="id",
                        y_col="breed",
                        has_ext=False,
                        subset="validation",
                        batch_size=1,
                        seed=42,
                        shuffle=True,
                        class_mode="categorical",
                        target_size=(image_size, image_size))

In [ ]:
STEP_SIZE_TRAIN=train_generator.n
STEP_SIZE_VALID=valid_generator.n

print(STEP_SIZE_TRAIN)
print(STEP_SIZE_VALID)

model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=3
)